In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,LabelEncoder

In [3]:
df = pd.read_csv("C:\\Users\\user\\Desktop\\data sets\\covid_toy.csv")

In [4]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [5]:
print(df['cough'].value_counts())
print(df['city'].value_counts())
df['gender'].value_counts()

Mild      62
Strong    38
Name: cough, dtype: int64
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: city, dtype: int64


Female    59
Male      41
Name: gender, dtype: int64

In [6]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(df.drop(columns = 'has_covid'),df['has_covid'],test_size=0.2,random_state=0)

In [8]:
x_train.head()

,age,gender,fever,cough,city
43,22,Female,99.0,Mild,Bangalore
62,56,Female,104.0,Strong,Bangalore
3,31,Female,98.0,Mild,Kolkata
71,75,Female,104.0,Strong,Delhi
45,72,Male,99.0,Mild,Bangalore


# Without columns Transfer

In [9]:
# Adding simple imputer to fever col
si = SimpleImputer()
x_train_fever = si.fit_transform(x_train[['fever']])

In [10]:
# also the test data
x_test_fever = si.fit_transform(x_test[['fever']])

x_train_fever.shape

(80, 1)

In [11]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
x_train_cough = oe.fit_transform(x_train[['cough']])

# also the test data
x_test_cough = oe.fit_transform(x_test[['cough']])

x_train_cough.shape

(80, 1)

In [12]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse=False,dtype=np.int32)
x_train_gender_city = ohe.fit_transform(x_train[['gender','city']])

# also the test data
x_test_gender_city = ohe.fit_transform(x_test[['gender','city']])

x_train_gender_city.shape

C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 4)

In [13]:
# Extracting Age
x_train_age = x_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
x_test_age = x_test.drop(columns=['gender','fever','cough','city']).values

x_train_age.shape

(80, 1)

In [14]:
x_train_transformed = np.concatenate((x_train_age,x_train_fever,x_train_gender_city,x_train_cough),axis=1)
# also the test data
x_test_transformed = np.concatenate((x_test_age,x_test_fever,x_test_gender_city,x_test_cough),axis=1)

pd.DataFrame(x_train_transformed,columns=('age','fever','Gender','benglore','delhi','mumbai','cough'))

,age,fever,Gender,benglore,delhi,mumbai,cough
0,22.0,99.0,0.0,0.0,0.0,0.0,0.0
1,56.0,104.0,0.0,0.0,0.0,0.0,1.0
2,31.0,98.0,0.0,0.0,1.0,0.0,0.0
3,75.0,104.0,0.0,1.0,0.0,0.0,1.0
4,72.0,99.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
75,51.0,101.0,0.0,0.0,1.0,0.0,1.0
76,65.0,99.0,1.0,0.0,0.0,0.0,0.0
77,42.0,104.0,1.0,0.0,0.0,1.0,0.0
78,18.0,104.0,0.0,0.0,0.0,0.0,0.0


In [15]:
x_train_transformed.shape

(80, 7)

# ColumnsTransformer

In [16]:
from sklearn.compose import ColumnTransformer

In [17]:
transformer = ColumnTransformer(transformers=[
    ('ab',SimpleImputer(),['fever']),
    ('bc',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('cd',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder = 'passthrough')


In [18]:
transformer.fit_transform(x_train).shape

C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 7)

In [19]:
transformer.transform(x_test).shape

(20, 7)